In [1]:
import pandas as pd
import json
from dateutil import parser
from datetime import datetime, timedelta

In [2]:
class RoleClass:
    def __init__(self,
                role_name,
                id,
                decisions_per_hour_per_stream):

        '''
        The role defines the decision-making capabilities of a 
        particular class of decision maker

        e.g. a Role could be 'Consultant majors'

        You may have >1 individual with the same role in an ED
        

        Params:
        -------

        role_name: str
            Name of role
            Examples: Cons Resus, Cons Majors, Cons Minors

        decisions_per_hour_per_stream: list of dicts
            List of dicts in the following format
            {'stream': str, 'decisions_per_hour': float}
            Where stream is the stream name
            Decisions per hour is 

            If a resource is able to make decisions for multiple streams,
            then the list should contain multiple dictionaries

        '''
    
        self.role_name = role_name
        self.id = id
        self.decisions_per_hour_per_stream = decisions_per_hour_per_stream 


def create_role_objects(user_session):
    queryset = Role.objects.filter(user_session=user_session)

    role_list = []

    for role_object in queryset:
        role_list.append(
            RoleClass(
                role_name = role_object.role_name,
                id = role_object.id,
                decisions_per_hour_per_stream = role_object.decisions_per_hour_per_stream
            )
        )

    return role_list

In [3]:
class RotaEntryClass:
    '''
    Object defining a week's worth of rota for a single
    individual
    '''

    def __init__(self,
                 id,
                 role,
                 core=True,
                 name=None,
                 prev_week=None,
                 monday=None,
                 tuesday=None,
                 wednesday=None,
                 thursday=None,
                 friday=None,
                 saturday=None,
                 sunday=None):
        '''
        role: RoleType object
            What role the rota entry relates to. 
            RoleType objects determine decisions per hour.

        core: boolean
            Whether a resource should be considered as core.
            False = resource is ad-hoc.


        name: str
            String giving name of resource e.g. if preferring to 
            work with actual names of individuals

        prev_week: ShiftType object 

        monday: ShiftType object

        tuesday: ShiftType object

        wednesday: ShiftType object

        thursday: ShiftType object

        friday: ShiftType object
        
        saturday: ShiftType object
        
        sunday: ShiftType object
        '''
        self.role = role
        self.core = core
        self.name = name
        self.id = id

        self.prev_week = prev_week
        self.monday = monday
        self.tuesday = tuesday
        self.wednesday = wednesday
        self.thursday = thursday
        self.friday = friday
        self.saturday = saturday
        self.sunday = sunday


def create_rota_objects(user_session):

    role_list = create_role_objects(user_session)
    shift_list = create_shift_objects(user_session)

    queryset = RotaEntry.objects.filter(user_session=user_session)

    def find_role_by_id(id):
        for role_object in role_list:
            if role_object.id == id:
                return role_object
    
    def find_shift_by_id(id):
        for shift_object in shift_list:
            if shift_object.id == id:
                return shift_object


    rota_entry_list = []

    for rota_object in queryset:
        rota_entry_list.append(
            RotaEntryClass(
                role = find_role_by_id(rota_object.role_type),
                core = rota_object.resource_type,
                name = rota_object.resource_name,
                id = rota_object.id,

                prev_week = find_shift_by_id(id),
                monday = find_shift_by_id(id),
                tuesday = find_shift_by_id(id),
                wednesday = find_shift_by_id(id),
                thursday = find_shift_by_id(id),
                friday = find_shift_by_id(id),
                saturday = find_shift_by_id(id),
                sunday = find_shift_by_id(id),
            )
        )

    return role_list, shift_list, rota_entry_list


In [4]:
class ShiftTypeClass:
    def __init__(self,
                 name,
                 id,
                 start_time,
                 end_time,
                 unavailability_1_start=None,
                 unavailability_1_end=None,
                 unavailability_2_start=None,
                 unavailability_2_end=None,
                 unavailability_3_start=None,
                 unavailability_3_end=None):
        '''
        Params:
        --------

        name: str
            Name of shift type
            (e.g. early, late, all day)

        start_time: str
            Time the shift begins
            Pass in the form HH:MM

        end_time: str
            Time the shift ends
            Pass in the form HH:MM

        unavailability_1_start: str (OPTIONAL)
            Time the first period of unavailability starts
            Pass in the form HH:MM

        unavailability_1_end: str (OPTIONAL)
            Time the first period of unavailability ends
            Pass in the form HH:MM

        unavailability_2_start: str (OPTIONAL)
            Time the second period of unavailability starts
            Pass in the form HH:MM            

        unavailability_2_end: str (OPTIONAL)
            Time the second period of unavailability ends
            Pass in the form HH:MM


        unavailability_3_start: str (OPTIONAL)
            Time the third period of unavailability starts
            Pass in the form HH:MM
            
        unavailability_3_end: str (OPTIONAL)
            Time the third period of unavailability ends
            Pass in the form HH:MM
        '''

        self.name = name
        self.id = id
        self.name_plottable = name.replace('_', ' ').title()

        self.start_time = self.try_datetime_parse(start_time)
        self.end_time = self.try_datetime_parse(end_time)

        self.unavailability_1_start = self.try_datetime_parse(unavailability_1_start)
        self.unavailability_1_end = self.try_datetime_parse(unavailability_1_end)

        self.unavailability_2_start = self.try_datetime_parse(unavailability_2_start)
        self.unavailability_2_end = self.try_datetime_parse(unavailability_2_end)

        self.unavailability_3_start = self.try_datetime_parse(unavailability_3_start)
        self.unavailability_3_end = self.try_datetime_parse(unavailability_3_end)

    def try_datetime_parse(self, time_string):
        if time_string not in [None, 'null', ' ']:
            return parser.parse(time_string).time()
        else:
            return None

    def decimal_time(self, time_of_interest):
        requested_time = getattr(self, time_of_interest)
        if requested_time is not None:
            return requested_time.hour + (requested_time.minute/60)
        else:
            return None

    def shift_type_dataframe(self):
        data = {
            'start_time': self.start_time,
            'end_time': self.end_time,
            'unavailability_1_start': self.unavailability_1_start,
            'unavailability_1_end': self.unavailability_1_end,
            'unavailability_2_start': self.unavailability_2_start,
            'unavailability_2_end': self.unavailability_2_end,
            'unavailability_3_start': self.unavailability_3_start,
            'unavailability_3_end': self.unavailability_3_end,
        }
        
        return pd.DataFrame.from_dict(
            data=data, 
            orient='index', 
            columns = [self.name]
            ) 


def create_shift_objects(user_session):
    queryset = Shift.objects.filter(user_session=user_session)

    shift_type_list = []

    for shift_object in queryset:
        shift_type_list.append(
            ShiftTypeClass(
                name = shift_object.shift_type_name,
                id = shift_object.id,
                start_time = shift_object.shift_start_time,
                end_time = shift_object.shift_end_time,
                unavailability_1_start = shift_object.break_1_start,
                unavailability_1_end = shift_object.break_1_end,
                unavailability_2_start = shift_object.break_2_start,
                unavailability_2_end = shift_object.break_2_end,
                unavailability_3_start = shift_object.break_3_start,
                unavailability_3_end = shift_object.break_3_end
            )
        )

    return shift_type_list

In [5]:
class StreamClass:
    def __init__(self, stream_name, id, stream_priority, time_for_decision):
        self.stream_name = stream_name
        self.id = id
        self.stream_priority = stream_priority
        self.time_for_decision = time_for_decision

# Create from json

In [6]:
def create_role_objects_from_json(role_json):

    role_list = []

    for role_object in role_json:
        role_list.append(
            RoleClass(
                role_name = role_object['role_name'],
                id = role_object['id'],
                decisions_per_hour_per_stream = role_object['decisions_per_hour_per_stream']
            )
        )

    return role_list

In [7]:
def create_shift_objects_from_json(shift_json):

    shift_type_list = []

    for shift_object in shift_json:
        shift_type_list.append(
            ShiftTypeClass(
                name = shift_object['shift_type_name'],
                id = shift_object['id'],
                start_time = shift_object['shift_start_time'],
                end_time = shift_object['shift_end_time'],
                unavailability_1_start = shift_object['break_1_start'],
                unavailability_1_end = shift_object['break_1_end'],
                unavailability_2_start = shift_object['break_2_start'],
                unavailability_2_end = shift_object['break_2_end'],
                unavailability_3_start = shift_object['break_3_start'],
                unavailability_3_end = shift_object['break_3_end']
            )
        )

    return shift_type_list

In [8]:
def create_rota_objects_from_json(shift_list, role_list, rota_json):

    def find_role_by_id(role):
        try:
            for role_object in role_list:
                if role_object.id == role['id']:
                    return role_object
        except TypeError:
            return None
    
    def find_shift_by_id(shift):
        try:
            for shift_object in shift_list:
                if shift_object.id == shift['id']:
                    return shift_object
        except TypeError:
            return None



    rota_entry_list = []

    for rota_object in rota_json:
        rota_entry_list.append(
            RotaEntryClass(
                role = find_role_by_id(rota_object['role_type']),
                core = rota_object['resource_type'],
                name = rota_object['resource_name'],
                id = rota_object['id'],

                prev_week = find_shift_by_id(rota_object['prev_week']),
                monday = find_shift_by_id(rota_object['monday']),
                tuesday = find_shift_by_id(rota_object['tuesday']),
                wednesday = find_shift_by_id(rota_object['wednesday']),
                thursday = find_shift_by_id(rota_object['thursday']),
                friday = find_shift_by_id(rota_object['friday']),
                saturday = find_shift_by_id(rota_object['saturday']),
                sunday = find_shift_by_id(rota_object['sunday']),
            )
        )

    return rota_entry_list

In [9]:
def create_stream_objects_from_json(stream_json):

    stream_list = []

    for stream_object in stream_json:
        stream_list.append(
            StreamClass(
                stream_name = stream_object['stream_name'],
                id = stream_object['id'],
                stream_priority = stream_object['stream_priority'],
                time_for_decision = stream_object['time_for_decision']
            )
        )

    return stream_list

# Create lists

## Streams

In [10]:
with open('get-historic-data-streams-from-db.json', 'r') as j:
     stream_json = json.loads(j.read())

In [11]:
stream_list = create_stream_objects_from_json(stream_json)

In [12]:
stream_list

## Roles

In [13]:
with open('own-role-types.json', 'r') as j:
     role_json = json.loads(j.read())

In [14]:
role_list = create_role_objects_from_json(role_json)

In [15]:
role_list

## Shifts

In [16]:
with open('own-shift-types.json', 'r') as j:
     shift_json = json.loads(j.read())

In [17]:
shift_list = create_shift_objects_from_json(shift_json)

In [18]:
shift_list

## Rotas

In [19]:
with open('own-rota-entries-detailed.json', 'r') as j:
     rota_json = json.loads(j.read())

In [20]:
rota_json 

[{'id': 4,
  'prev_week': None,
  'monday': None,
  'tuesday': None,
  'wednesday': None,
  'thursday': None,
  'friday': {'id': 9,
   'user_session': 'vcg4ebb7rk5wfunt0y4iasnz58mdurvm',
   'shift_type_name': 'Late',
   'shift_start_time': '2021-01-01T18:00:00.000Z',
   'shift_end_time': '2021-01-01T02:00:00.000Z',
   'break_1_start': 'null',
   'break_1_end': 'null',
   'break_2_start': 'null',
   'break_2_end': 'null',
   'break_3_start': 'null',
   'break_3_end': 'null'},
  'saturday': None,
  'sunday': None,
  'role_type': {'id': 11,
   'user_session': 'vcg4ebb7rk5wfunt0y4iasnz58mdurvm',
   'role_name': 'Band 5',
   'decisions_per_hour_per_stream': [{'stream_name': 'Resus',
     'stream_object_id': 201,
     'decisions_per_hour': 0},
    {'stream_name': 'Majors',
     'stream_object_id': 199,
     'decisions_per_hour': 0},
    {'stream_name': 'Minors',
     'stream_object_id': 200,
     'decisions_per_hour': '0.5'}]},
  'user_session': 'vcg4ebb7rk5wfunt0y4iasnz58mdurvm',
  'resourc

In [21]:
rota_entry_list = create_rota_objects_from_json(shift_list, role_list, rota_json)

In [22]:
rota_entry_list

# Calculate hourly capacity


In [23]:
start_date = datetime(2021, 7, 16)

In [24]:
datetime_index = pd.date_range(start_date - timedelta(days=1),
                               start_date + timedelta(days=7),
                               freq='h')[:-1]

In [37]:
# Set up empty dictionaries that will be used to record key:value pairs
# of stream: data
overall_decisions_per_hour = {}
overall_resources = {}

for resource in rota_entry_list:
    print(f"Working on resource {resource.name}")

    # Set up empty lists for storing stream-level data
    stream_decisions_per_hour = {}
    stream_resources = {}

    # Iterate through the streams
    streams_in_data_names = [stream.stream_name for stream in stream_list]

    for stream in streams_in_data_names:
        # Create lists for storing an individual day's data
        decisions_per_hour = []
        resources = []
        
        # Get the number of decisions that can be made per hour for this stream
        decisions_per_hour_for_stream = float(
            # Get the relevant item for the stream
            next((item 
                  for item 
                  in resource.role.decisions_per_hour_per_stream 
                  if item["stream_name"] == stream), 
                  # Default to return if not found (should always be found, but worth handling)
                  None)
                  # Grab just the decisions_per_hour data for that stream
                  ['decisions_per_hour']
        ) 

        # Check that it's worth continuing with calculations at this stage
        # for this stream for this resource (i.e. that they will not just
        # be zero for the whole week)
        if (decisions_per_hour_for_stream is not None) \
            and (decisions_per_hour_for_stream != 0):
            print(f"Working on stream {stream} with {decisions_per_hour_for_stream} decision(s) per hour")

            # For the first day, the hours since midnight worked will be zero
            hours_from_midnight_already_worked = None

            # Begin iterating through days of the week, including 'prev_week'
            # which is used to account for any overnight shifts that finish on
            # Monday 
            for day in ['prev_week', 'monday', 'tuesday', 
                        'wednesday', 'thursday', 'friday', 
                        'saturday', 'sunday']:
                
                # Get shift data for the resource on this day
                shift = (getattr(resource, day))

                if shift is None:
                    # Create a list with '0' for the full day
                    decisions_per_hour.extend([0 for i in range(24)])
                    resources.extend([0 for i in range(24)])
                
                if shift is not None:
                    print(f"Working on {day} shift")

                    # Add the appropriate number of zeros,
                    # for hours not worked, then continue on
                    print(f"Start time for shift: {shift.start_time.hour}:00")
                    print(f"End time for shift: {shift.end_time.hour}:00")
                    if hours_from_midnight_already_worked == None:
                        # Work out how many hours to append 0 for
                        empty_hours = shift.start_time.hour - 1
                    else:
                        empty_hours = shift.start_time.hour - 1 - hours_from_midnight_already_worked
                    
                    decisions_per_hour.extend([0 for i in range(empty_hours)])
                    resources.extend([0 for i in range(empty_hours)])
                    
                    # ---- Deal with the hours that decision-making capability will be available in ---- #
                    
                    # Check whether fractional capacity needs to be added for shifts that don't START on the hour
                    if shift.start_time.hour % 1 != 0:
                        decisions_per_hour.extend([(shift.start_time.hour % 1) * decisions_per_hour_for_stream])
                        resources.extend([shift.start_time.hour % 1])
                    
                    # If shift doesn't span midnight
                    if shift.start_time < shift.end_time:
                        if shift.start_time.hour % 1 != 0:
                            # Deduct one hour from the working hours to account for the fraction
                            working_hours = (shift.end_time.hour - shift.start_time.hour) - 1
                        else: 
                            # If no fractional start time, working hours are just the total number of hours
                            working_hours = (shift.end_time.hour - shift.start_time.hour)
                    # If shift does span midnight:
                    elif shift.start_time > shift.end_time:
                        working_hours = (24 - shift.start_time.hour) + shift.end_time.hour
                    
                    
                    # Append data for the number of working hours
                    decisions_per_hour.extend([decisions_per_hour_for_stream for i in range(working_hours)])
                    print(f"Dph: {[decisions_per_hour_for_stream for i in range(working_hours)]}") 
                    resources.extend([1 for i in range(working_hours)])
                    print(f"Resources: {[1 for i in range(working_hours)]}") 
                    
                    # Check whether fractional capacity needs to be added for shifts that don't END on the hour
                    if shift.end_time.hour % 1 != 0:
                        decisions_per_hour.extend((shift.end_time.hour % 1) * decisions_per_hour_for_stream)
                        resources.extend((shift.end_time.hour % 1))

                    # If shift doesn't span midnight
                    if shift.start_time < shift.end_time:
                        # Fill in the remaining hours until midnight with zeros
                        remaining_hours = 24 - shift.end_time.hour
                        decisions_per_hour.extend([0 for i in range(remaining_hours)])
                        resources.extend([0 for i in range(remaining_hours)])

                        hours_from_midnight_already_worked = None

                    # If shift start time is bigger than shift end time, this implies shift spans midnight, so the next day
                    # will need to be dealt with slightly differently
                    elif shift.start_time > shift.end_time:
                        if (shift.end_time.hour % 1) != 0:  
                            hours_from_midnight_already_worked = shift.end_time.hour + 1
                        else:
                            hours_from_midnight_already_worked = shift.end_time.hour
                              



            
                    # print(shift.end_time.hour)

        else:
            decisions_per_hour = [0 for i in range(24*8)]
            resources = [0 for i in range(24*8)]

        # TODO: Check indentation
        stream_decisions_per_hour[stream] = decisions_per_hour
        if len(decisions_per_hour) != 192:
            print(f"Length of decisions_per_hour: {len(decisions_per_hour)}")
        
        stream_resources[stream] = resources
        if len(decisions_per_hour) != 192:
            print(f"Length of resources: {len(decisions_per_hour)}")

    print("---------")
    print(" ")

    overall_decisions_per_hour[resource.name] = stream_decisions_per_hour
    overall_resources[resource.name] = stream_resources

Working on resource Dr Carling
Working on stream Minors with 0.5 decision(s) per hour
Working on friday shift
Start time for shift: 18:00
End time for shift: 2:00
Dph: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Resources: [1, 1, 1, 1, 1, 1, 1, 1]
Length of decisions_per_hour: 193
Length of resources: 193
---------
 
Working on resource Dr Jones
Working on stream Resus with 1.0 decision(s) per hour
Working on tuesday shift
Start time for shift: 6:00
End time for shift: 16:00
Dph: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Resources: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Working on wednesday shift
Start time for shift: 9:00
End time for shift: 18:00
Dph: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Resources: [1, 1, 1, 1, 1, 1, 1, 1, 1]
Working on saturday shift
Start time for shift: 6:00
End time for shift: 16:00
Dph: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Resources: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Length of decisions_per_hour: 189
Length of resources: 189
---------
 
Working

## Calculate total available resources by role type

In [33]:
overall_resources

{'Dr Carling': {'Resus': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,

## Dealing with flexible capacity

In order of stream priority:
- if available dedicated capacity is sufficient for forecast demand, try giving glexible capacity to the next stream down.
- if available dedicated capacity is insufficient then give it to the next stream down in priority (that it can share with), though make sure to account for differences in decisiosn per hour for different streams by the same role.
- if the lowest priority stream doesn't need it, then move any remaining flexible capacity back into the highest priority stream's pot. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a56ed435-0fca-4e33-b393-07cccaf98df7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>